Load All the Initial Python Modules
 - astra_ctvlib is the core component that performs the GPU-tomography reconstuctions

In [ ]:
import Utils.astra_ctvlib as astra_ctvlib
import Utils.pytvlib as pytvlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import h5py

In [ ]:
# Auxilary Function to Visualize Slices of the Phantom Object or Reconstruction

def display_recon_slices(inVolume):

    fig, ax = plt.subplots(1,3,figsize=(25,25))
    ax = ax.flatten()
    ax[0].imshow(inVolume[:,],cmap='gray'); ax[0].axis('off')
    ax[1].imshow(inVolume[:,],cmap='gray'); ax[1].axis('off')
    ax[2].imshow(inVolume[:,],cmap='gray'); ax[2].axis('off')

### Simulating Projections 

Load the Input Dataset (commonly referered to as Tilt Series)

For this demo we'll show how we can use tomo_TV to generate these projections images from an input volume. 

For experimental reconstructions, skip this step and go straight to the reconstruction portion. 

In [ ]:
# Load the Original Volume
file = h5py.File('STO_nanocubes.h5','r')
original_volume = file['STO'][:]
file.close()

# Read Dimensions of Test Object
(Nslice, Nray, _) = original_volume.shape
# In cases where projection images aren't square: Nslice != Nray

# Visualize Slices of the Phantom Object
fig, ax = plt.subplots(1,3,figsize=(25,25))
ax = ax.flatten()
ax[0].imshow(original_volume[:,])
ax[1].imshow(original_volume[:,])
ax[2].imshow(original_volume[:,])

Before Accessing any of the tomography functionality, we have to initialize the core class and any sub-functions that we would like to interface with.

To inialize this class we need to provide two parameters, 
1. The # of pixels of the projection images / volumes 

2. The tilt angles we would like to simulate the experiment, or angles that projection images were collected. 

In [ ]:
# For this demo, we'll use conventional electron tomography experimental parameters: 
# ±70º with a +2º tilt increment. 
tiltAngles = np.arange(-70,72,2) 

# Initialize the C++ Object..
tomo = astra_ctvlib.astra_ctvlib(Nslice, Nray, np.deg2rad(tiltAngles))

# astra_ctvlib by default creates one 3D volume for the reconstruction, 
# any additional volumes needs to be externally intialized 
# (this is to save memory consumption)
tomo.initialize_initial_volume()

In [ ]:
# Let's pass the volume from python to C++  
for s in range(Nslice):
    tomo.set_original_volume(original_volume[s,:,:],s)

# Now Let's Create the Projection Images
tomo.create_projections()

# Optional: Apply poisson noise to volume.
SNR = 10
if SNR != 0: tomo.poisson_noise(SNR)

# Return the projections to Python
Nangles = tiltAngles.shape[0]
tiltSeries = np.zeros([Nslice, Nray, Nangles],dtype=np.float32)
projections = tomo.get_projections()

for i in range(Nangles):
    tiltSeries[:,:,i] = projections[:,Nray*i:Nray*(i+1)]

# Visualize Tilt Series
plt.figure(figsize=(25,25))
fig, ax = plt.subplots(1,3,figsize=(25,25))
ax = ax.flatten()
for j in range(3): ax[j].imshow(tiltSeries[:,:,j],cmap='gray'); ax[j].axis('off'); ax[j].title('Tilt Angle: ', tiltAngles[j])

### Tomography Recontructions

Now that we have a sample tilt series, let's actually reconstruct the data and see how it compares to the ground truth

In [ ]:
# We have quite a few algorithms to choose from, for simplicitiy 
# let's start off with FBP

# Specify # of iterations for the algorith to run
Niter = 150

tomo.initialize_FBP()

# Main Loop
tomo.FBP()

filter = 'ram-lak'
tomo.initialize_FBP(filter)

# Reconstruct
tomo.FBP()
    
# Return the Reconstruction to Python
recon = np.zeros([Nslice,Nray,Nray],dtype=np.float32)
for s in range(Nslice):
    recon[s,] = tomo.get_recon(s)
    
# Visualize Slices of the Phantom Object
plt.figure()

Oof, as expected FBP is terrible. Let's now try an iterative algorithm like SIRT. 

In [ ]:
tomo.initialize_SIRT()

# Main Loop
for i in tqdm(range(Niter)):
    tomo.SIRT()
    
# Return the Reconstruction to Python
recon = np.zeros([Nslice,Nray,Nray],dtype=np.float32)
for s in range(Nslice):
    recon[s,] = tomo.get_recon(s)
    
# Visualize Slices of the Phantom Object
plt.figure()

Wow! So much better. We can also add regularization and try clean up the reconstructions further